In [ ]:
def process_assembly(assembly_code):
    address = 0x00400000  # Địa chỉ bắt đầu
    instructions = []  # Danh sách các lệnh với địa chỉ
    labels = {}  # Lưu địa chỉ của các nhãn

    for line in assembly_code:
        # Loại bỏ khoảng trống đầu cuối và phần chú thích
        line = line.split('#')[0].strip()
        if not line:  # Nếu dòng trống (hoặc chỉ chứa chú thích), bỏ qua
            continue

        if ':' in line:  # Nếu có nhãn (label)
            label, *rest = line.split(':', 1)  # Tách nhãn và phần còn lại
            label = label.strip()
            labels[label] = address  # Ghi lại địa chỉ của nhãn
            if rest:  # Nếu sau nhãn còn lệnh, xử lý tiếp
                rest_line = rest[0].strip()
                if rest_line:
                    instructions.append((address, rest_line))
                    address += 4
        else:  # Nếu chỉ là lệnh, không có nhãn
            instructions.append((address, line))
            address += 4

    return instructions, labels


# Ví dụ input
assembly_code = [
    "label1: add $t1, $t2, $t3 # Cộng t1 = t2 + t3",
    "   sub $t4, $t5, $t6    # Trừ t4 = t5 - t6",
    "label2:   # Đây là một nhãn",
    "     mul $t7, $t8, $t9   ",
    "# Một dòng chỉ có chú thích",
    "div $t0, $t1, $t2  ",
]

# Xử lý đoạn mã
instructions, labels = process_assembly(assembly_code)

# Output
print("Instructions with addresses:")
for addr, instr in instructions:
    print(f"{hex(addr)}: {instr}")

print("\nLabels with addresses:")
for label, addr in labels.items():
    print(f"{label}: {hex(addr)}")


In [ ]:
def DeleteComment(code):
    cleaned_code = []
    for line in code:
        line = line.split('#')[0].strip()
        if line:
            cleaned_code.append(line)
    return cleaned_code
def SourceToBasic(code):
    converted_code = []
    for line in code:
        line = line.strip()
        
        # Kiểm tra xem dòng có nhãn ở đầu hay không
        label = ""
        if ":" in line:
            label_part = line.split(":")
            label = label_part[0].strip()  # Lấy nhãn
            line = label_part[1].strip()  # Phần còn lại là mã lệnh
            
        # Kiểm tra nếu lệnh là `beq`
        if line.startswith("beq"):
            parts = line.split(',')
            if len(parts) == 3:
                reg1 = parts[0].split()[1]  # Thanh ghi đầu tiên
                reg2 = parts[1].strip()    # Thanh ghi thứ hai hoặc giá trị so sánh
                target_label = parts[2].strip()  # Nhãn nhảy

                # Thay thế bằng hai lệnh: addi và beq
                if label:
                    converted_code.append(f"{label}:")  # Thêm nhãn trước
                converted_code.append(f"addi $at, $zero, {reg2}")  # addi $at, $zero, giá trị so sánh
                converted_code.append(f"beq $at, {reg1}, {target_label}")  # beq $at, reg1, nhãn
            else:
                if label:
                    converted_code.append(f"{label}: {line}")  # Giữ nguyên nếu cú pháp không đúng
                else:
                    converted_code.append(line)
        else:
            if label:
                converted_code.append(f"{label}: {line}")  # Dòng khác kèm nhãn
            else:
                converted_code.append(line)  # Các lệnh khác giữ nguyên
    return converted_code


def ProcessAsmFile(input_file, output_file):
    with open(input_file, 'r') as infile:
        code = infile.readlines()

    # Xóa comment
    cleaned_code = DeleteComment(code)

    # Chuyển đổi sang dạng basic
    basic_code = SourceToBasic(cleaned_code)

    # Ghi file đầu ra
    with open(output_file, 'w') as outfile:
        outfile.write('\n'.join(basic_code))

# Sử dụng
input_asm_file = "test.asm"   # Đường dẫn file MIPS Assembly gốc
output_asm_file = "test_basic.asm" # Đường dẫn file MIPS Assembly đã chuyển đổi
ProcessAsmFile(input_asm_file, output_asm_file)